In [5]:
import numpy as np
import pandas as pd

In [9]:
#data = np.genfromtxt("alturas-edades.tsv", delimiter="\t")
#data

In [16]:
data = pd.read_csv("alturas-edades.tsv", delimiter="\t")
# Obtener la media para corregir los datos faltantes
data.Edad = data.Edad.fillna(data.Edad.mean())
data.head()

,Estatura,Edad
0,1.77,27.5
1,1.74,27.5
2,1.72,24.0
3,1.78,27.5
4,1.65,32.0


In [18]:
data.values.T

array([[ 1.77,  1.74,  1.72,  1.78,  1.65,  1.66,  1.64,  1.85,  1.85,
         1.75,  1.73,  1.79,  1.82,  1.65,  1.79,  1.72,  1.7 ,  1.57,
         1.66,  1.75],
       [27.5 , 27.5 , 24.  , 27.5 , 32.  , 27.5 , 27.5 , 27.5 , 26.  ,
        27.5 , 27.5 , 27.5 , 27.5 , 27.5 , 27.5 , 34.  , 26.  , 25.  ,
        26.  , 27.  ]])

In [19]:
np.cov(data.values.T)

array([[ 5.60500000e-03, -1.57894737e-03],
       [-1.57894737e-03,  4.63157895e+00]])

In [26]:
np.mean(data.values, axis=0)

array([ 1.7295, 27.5   ])

In [25]:
from scipy.stats import multivariate_normal
rv = multivariate_normal()

array([[ 1.77, 27.5 ],
       [ 1.74, 27.5 ],
       [ 1.72, 24.  ],
       [ 1.78, 27.5 ],
       [ 1.65, 32.  ],
       [ 1.66, 27.5 ],
       [ 1.64, 27.5 ],
       [ 1.85, 27.5 ],
       [ 1.85, 26.  ],
       [ 1.75, 27.5 ],
       [ 1.73, 27.5 ],
       [ 1.79, 27.5 ],
       [ 1.82, 27.5 ],
       [ 1.65, 27.5 ],
       [ 1.79, 27.5 ],
       [ 1.72, 34.  ],
       [ 1.7 , 26.  ],
       [ 1.57, 25.  ],
       [ 1.66, 26.  ],
       [ 1.75, 27.  ]])